In [1]:
from collections import Counter
from operator import itemgetter
import copy

def remove_tag(training_file): # generate train_no_tag file
    f = open(training_file,"r",encoding="utf8")
    new_file = training_file+"_perceptron_100"
    f1 = open(new_file,"w+",encoding="utf8")
    for line in f:
        line = line.split()
        if len(line)>1:
            f1.write(line[0]+"\n")
        else:
            f1.write("\n")

    f.close()
    f1.close()
    return

In [2]:
def para(training_file): # output a dictionary of emission_parameter
    cnt_y = Counter() # count the y tags
    e_para = Counter() # count the y|x tags
    t_para = Counter()
    state = "start"
    y0_y1 = ""
    # init_e_para = Counter()
    # init_t_para = Counter()
    f = open(training_file, "r", encoding='utf8')
    for line in f:
        line = line.rstrip('\n')
        if (len(line) == 0):
            y0_y1 = state + " stop"
            state = "stop"
        else:
            e_para[line]+=1
            line = line.split()[1]
            y0_y1 = state + " " + line
            state = line
            cnt_y[line]+=1
        t_para[y0_y1]+=1

    for tag in cnt_y:
        e_para["new_word "+tag]=1

    f.close()
    return  e_para, t_para

def init_para(training_file):
    cnt_y = Counter() # count the y tags
    e_para = Counter() # count the y|x tags
    t_para = Counter()
    state = "start"
    y0_y1 = ""
    # init_e_para = Counter()
    # init_t_para = Counter()
    f = open(training_file, "r", encoding='utf8')
    for line in f:
        line = line.rstrip('\n')
        if (line == ""):
            y0_y1 = state + " stop"
            state = "stop"
        else:
            e_para[line]=0
            line = line.split()[1]
            y0_y1 = state + " " + line
            state = line
            cnt_y[line]=0
        t_para[y0_y1]=0

    for tag in cnt_y:
        e_para["new_word "+tag]=0

    f.close()
    return  e_para, t_para

In [3]:
def test_phase_perceptron(test_file,emission_parameter,transition_parameter,top_k=1, run = 1):
    tag = ["O","B-neutral","B-positive","B-negative","I-neutral","I-positive","I-negative"]
    step_k = 1
    v_label = "O"
    u_label = "start"
    optimal_value = -1
    pi = [step_k,v_label,optimal_value,u_label]
    sentence = []
    output = []
    list_u = [0,1,2,3,4,5,6]
    dict_u = {}
    state = (step_k, list_u)
    data_x = set()
    for entry in emission_parameter.keys():
        entry = entry.split(' ')[0]
        data_x.add(entry)


    file_dir = test_file[:-3]+str(int(test_file[-3:])+run)
    f = open(test_file, "r", encoding='utf8')
    f1 = open(file_dir, "w+", encoding='utf8')

    for line in f:
        line = line.rstrip()
        if line!="":
            for id_v,v in enumerate(tag):
                best = -1
                best_add = -1
                for id_u,u in enumerate(state[1]): #enumerate list_u stored in state
                    if(line in data_x):
                        b_word = line + " " + v
                        b_value = emission_parameter[b_word]
                    else:
                        b_word = "new_word " + v
                        b_value = emission_parameter[b_word]

                    if state[0] == 1:
                        u_label = "start"
                        uv_label = u_label+" " + v
                        a_uv_value = transition_parameter[uv_label]
                        optimal_value = a_uv_value+b_value

                    else:
                        u_label = u[1]
                        uv_label = u_label + " " + v
                        a_uv_value = transition_parameter[uv_label]
                        optimal_value =  u[2] + a_uv_value + b_value

                    if optimal_value>best:
                        best = optimal_value
                        pi = [step_k, v, best, u_label]

                dict_u[(step_k,v)] = pi[3]
                list_u[id_v] = pi

            sentence.insert(0,line)
            list_u2 = copy.deepcopy(list_u)
            step_k+=1
            state = (step_k,list_u2)

        else: # when it reaches the empty line
            best = -1
            for id_u,u in enumerate(state[1]):
                uv_label = u[1] + " stop"
                a_uv_value = transition_parameter[uv_label]
                optimal_value = u[2]+a_uv_value
                pi = [state[0],"stop",optimal_value,u[1]]
                list_u[id_u] = pi


            list_u = sorted(list_u,key=itemgetter(2),reverse=True)  # arrange by descending order to choose top_k
            u_label = list_u[top_k-1][3]
            output.append(sentence[0]+" "+u_label+"\n")

            for i in range(1,len(sentence)): # back tracking to find the y_labels
                search_key = (step_k-i,u_label)
                u_label = dict_u[search_key]
                output.insert(0,sentence[i]+" "+u_label+"\n")

            for i in output:
                f1.write(i)
            f1.write("\n")
            sentence = []
            output=[]
            dict_u={}
            step_k=1
            state = (step_k,[0,1,2,3,4,5,6])
    f1.close()
    return 


In [4]:
def update_para1(training_file, n = 3):
    parameter1 = para(training_file)
    e_c1 = copy.deepcopy(parameter1[0])
    t_c1 = copy.deepcopy(parameter1[1])
    init_parameter = init_para(training_file) #initialize epara and tpara to 0
    e_para = copy.deepcopy(init_parameter[0])
    t_para = copy.deepcopy(init_parameter[1])

    remove_tag(training_file) # generate training file without tags
    blank_file = training_file+"_perceptron_100"
    for run_num in range(n):
        print("Running %d of %d:" %(run_num+1,n) )
        test_phase_perceptron(blank_file,e_para,t_para,run = run_num+1) # generates prediction file
        predicted_file = blank_file[:-3]+str(int(blank_file[-3:])+run_num+1)
        parameter2 = para(predicted_file) # to get e_c2, t_c2
        e_c2 = copy.deepcopy(parameter2[0])
        t_c2 = copy.deepcopy(parameter2[1])
        
        # follow formula - e_para = e_para+c1-c2
        e_para += e_c1  
        e_para.subtract(e_c2)

        t_para += t_c1
        t_para.subtract(t_c2)

    return e_para ,t_para

In [5]:
def update_para2(training_file,e_para,t_para, n = 3):
    parameter1 = para(training_file)
    e_c1 = copy.deepcopy(parameter1[0])
    t_c1 = copy.deepcopy(parameter1[1])

    remove_tag(training_file) # generate training file without tags
    blank_file = training_file+"_perceptron_100"
    for run_num in range(n):
        print("Running %d of %d:" %(run_num+1,n) )
        test_phase_perceptron(blank_file,e_para,t_para,run = run_num+1)
        predicted_file = blank_file[:-3]+str(int(blank_file[-3:])+run_num+1)
        parameter2 = para(predicted_file)
        e_c2 = copy.deepcopy(parameter2[0])
        t_c2 = copy.deepcopy(parameter2[1])

        e_para += e_c1
        e_para.subtract(e_c2)

        t_para += t_c1
        t_para.subtract(t_c2)

    return e_para ,t_para

In [6]:
training_file = "D:\\Downloads\\School Stuff\\Term 6 Sep-Dec 16\\Machine Learning\\Project\\EN\\train"
test_file = "D:\\Downloads\\School Stuff\\Term 6 Sep-Dec 16\\Machine Learning\\Project\\EN\\dev.in"
parameter = update_para1(training_file, n = 3)
print (parameter[1].most_common(10))
print(parameter[0].most_common(10))
e_para = parameter[0]
t_para = parameter[1]


Running 1 of 3:
Running 2 of 3:
Running 3 of 3:
[('O O', 7410), ('I-neutral O', 1398), ('B-neutral I-neutral', 954), ('O stop', 944), ('O B-positive', 936), ('B-positive O', 594), ('I-neutral I-neutral', 594), ('B-positive I-positive', 504), ('I-positive O', 468), ('O B-negative', 276)]
[(', O', 230), ('the O', 201), ('. O', 187), ('to O', 182), (': O', 161), ('Facebook B-neutral', 145), ('I O', 125), ('! O', 117), ('... O', 115), ('a O', 113)]


In [8]:
e_para = parameter[0]
t_para = parameter[1]
parameter = update_para2(training_file, e_para, t_para,n=10)

print (parameter[1].most_common(10))
print(parameter[0].most_common(10))

Running 1 of 10:
Running 2 of 10:
Running 3 of 10:
Running 4 of 10:
Running 5 of 10:
Running 6 of 10:
Running 7 of 10:
Running 8 of 10:
Running 9 of 10:
Running 10 of 10:
[('O O', 9347), ('O B-positive', 3656), ('B-neutral I-neutral', 3108), ('I-neutral O', 2796), ('B-positive O', 2574), ('I-neutral I-neutral', 2517), ('B-positive I-positive', 2184), ('I-positive O', 2028), ('O B-negative', 1196), ('B-negative O', 936)]
[(', O', 650), ('Facebook B-neutral', 646), ('the O', 602), ('to O', 532), (': O', 492), ('... O', 415), ('a O', 374), ('. O', 370), ('I O', 340), ('and O', 337)]


[('O O', 25720), ('O B-positive', 14032), ('B-positive I-positive', 13562), ('B-neutral I-neutral', 12138), ('I-neutral I-neutral', 11847), ('B-negative I-negative', 11593), ('I-positive O', 11473), ('B-neutral O', 11179), ('I-negative O', 10793), ('I-positive I-positive', 9997)]
[('Facebook B-neutral', 13956), ('Twitter B-neutral', 8339), ('News I-neutral', 4461), ('Google B-neutral', 4406), (': O', 4201), ('to O', 3887), (', O', 3804), ('is O', 3609), ('the O', 3607), ('... O', 3589)]
